In [97]:
# Exercise 6.10: Stochastic Wind Re-solve the windy gridworld task with King’s
# moves, assuming that the effect of the wind, if there is any, is stochastic, sometimes
# varying by 1 from the mean values given for each column. That is, a third of the
# time you move exactly according to these values, as in the previous exercise, but also
# a third of the time you move one cell above that, and another third of the time you
# move one cell below that. For example, if you are one cell to the right of the goal
# and you move left, then one-third of the time you move one cell above the goal,
# one-third of the time you move two cells above the goal, and one-third of the time

import numpy as np

#set up the board
wind_list = [0,0,0,1,1,1,2,2,1,0]
action_list = [(1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)]
board = {}
board_width = 10
board_height = 7
for i in range(board_width):
    for j in range(board_height):
        board[(i,j)] = 0

#can only choose actions that let you remain on the board
def get_random_action(state,board,action_space):
    while True:
        ret_value = action_space[np.random.randint(0,len(action_space))]
        temp_value = (ret_value[0] + state[0],ret_value[1] + state[1])
        if board.get(temp_value,-19) != -19:
            break
    return ret_value

def get_max_action_value(state,board,action_space,q_table):
    max_value = -19
    tie_list = []
    for i in action_list:
        #print( "i is " + str(i) )
        temp_value = (i[0] + state[0],i[1] + state[1])
        
        if board.get(temp_value,-19) == -19:
            #print("can't use that value, out of range" + str(temp_value))
            continue
            
        if q_table.get(((state,i)),0) > max_value:
            max_value = q_table.get(((state,i)),0)
            max_action = i
            tie_list = []
            tie_list.append(i)
        elif q_table.get(((state,i)),0) == max_value:
            tie_list.append(i)
            #print("max action is" + str(max_action))
    #print("max action is" + str(max_action))
    if len(tie_list) > 1:
        max_action = tie_list[np.random.randint(0,len(tie_list))]
    return max_action,max_value       

def get_wind_state(state,board,action,wind_list):
    wind_push = wind_list[state[0]]
    if wind_push > 0:
        wind_push += wind_push + np.random.randint(-1,2) #add -1,0, or 1 to the wind
    
    temp_height = state[1] + action[1] + wind_push
    if temp_height >= board_height:
        temp_height = board_height - 1
        
    temp_state = (state[0] + action[0], temp_height)
    #print(temp_state)
    return temp_state

In [161]:
#using Q learning        
episodes = 1000
goal_reward = 1.
alpha = .25
gamma = 0.99#1.#0.95
e_max = 1.
e_min = .01
e_decay = (e_max-e_min)/episodes
steps = -1
goal_state = (7,3)
start_state = (0,3)
Q = {}
step_cost = -.0005 #0. #-.001

for i in range(episodes):
    steps += 1
    state = start_state
    j = 0
    while True:
        j += 1
        explore_p = e_max - steps*e_decay
        #print(explore_p,e_max,steps,e_decay)
        #e greedy on policy
        if np.random.rand() <= explore_p:
            action = get_random_action(state,board,action_list)
            #print("action is random " + str(action) )
        else:
            action,_ = get_max_action_value(state,board,action_list,Q)
            #print("step " + str(j) + " action is greedy" + str(action))
            
        next_state = get_wind_state(state,board,action,wind_list)
        
        if next_state == goal_state:
            Q[(state,action)] = Q.get((state,action),0.) + alpha*(goal_reward + gamma*0 - Q.get((state,action),0.))
            print( "Episode {}. reached goal in steps: {}".format(i,j))
            break
        else:
            _,next_state_value = get_max_action_value(next_state,board,action_list,Q)
            Q[(state,action)] = Q.get((state,action),0.) + alpha*(step_cost + gamma*next_state_value - Q.get((state,action),0.))
            state = next_state
#         if j % 10000 == 0:
#             print(j,state,next_state)
#             #print(Q)
#             #break
    if episodes % (episodes/10) == 0:
        alpha = alpha * .9
    #break

Episode 0. reached goal in steps: 183
Episode 1. reached goal in steps: 11929
Episode 2. reached goal in steps: 2352
Episode 3. reached goal in steps: 10548
Episode 4. reached goal in steps: 5862
Episode 5. reached goal in steps: 2364
Episode 6. reached goal in steps: 4141
Episode 7. reached goal in steps: 116
Episode 8. reached goal in steps: 5722
Episode 9. reached goal in steps: 18134
Episode 10. reached goal in steps: 4092
Episode 11. reached goal in steps: 5781
Episode 12. reached goal in steps: 8539
Episode 13. reached goal in steps: 1858
Episode 14. reached goal in steps: 11622
Episode 15. reached goal in steps: 27174
Episode 16. reached goal in steps: 244
Episode 17. reached goal in steps: 17705
Episode 18. reached goal in steps: 2531
Episode 19. reached goal in steps: 3916
Episode 20. reached goal in steps: 7308
Episode 21. reached goal in steps: 15012
Episode 22. reached goal in steps: 897
Episode 23. reached goal in steps: 6795
Episode 24. reached goal in steps: 6089
Episode

In [162]:
print(len(Q))

391


In [163]:
import operator
#x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
#sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_q = sorted(Q.items(),key=operator.itemgetter(1))
print(sorted_q)

[(((5, 2), (1, 0)), -8.201250000000002e-05), (((5, 2), (0, -1)), -8.201250000000002e-05), (((5, 1), (0, -1)), -3.5303692060125025e-05), (((9, 1), (0, -1)), -1.4855202751546435e-05), (((7, 1), (1, -1)), 4.99484898461331e-17), (((7, 2), (-1, 0)), 4.746950833988178e-11), (((8, 0), (1, 1)), 7.102735317653911e-11), (((8, 0), (1, 0)), 1.3331454282050376e-08), (((6, 4), (1, 0)), 2.1532267289284936e-07), (((7, 2), (-1, 1)), 5.471702022365809e-07), (((7, 2), (1, 0)), 5.679859523132378e-07), (((8, 1), (1, -1)), 9.581133806778068e-06), (((7, 1), (0, -1)), 1.0367710188768045e-05), (((9, 0), (0, 1)), 2.4493062223517128e-05), (((8, 1), (1, 1)), 6.594023223658217e-05), (((6, 4), (-1, -1)), 7.611525791496852e-05), (((9, 0), (-1, 0)), 8.246969001629921e-05), (((5, 1), (0, 1)), 0.0001264982189958268), (((8, 0), (0, 1)), 0.00021259521759414297), (((4, 1), (1, -1)), 0.0006183787152225739), (((6, 3), (-1, -1)), 0.0008012005027590651), (((7, 2), (-1, -1)), 0.0008515789124482045), (((5, 3), (-1, 0)), 0.00142

In [166]:
#optimal path, keeping wind stochastic but optimal path depends on how the wind blows

def get_constant_wind_state(state,board,action,wind_list):
    wind_push = wind_list[state[0]]
    if wind_push > 0:
        wind_push += wind_push + np.random.randint(-1,2) #add -1,0, or 1 to the wind
    
    temp_height = state[1] + action[1] + wind_push
    if temp_height >= board_height:
        temp_height = board_height - 1
        
    temp_state = (state[0] + action[0], temp_height)
    #print(temp_state)
    return temp_state

state = (0,3)
steps = 0
while state != goal_state:
    if steps > 30:
        break
    steps += 1
    max_value = -19
    max_action = None
    for i in action_list:
        #print((state,action))
        temp_value = Q.get((state,i),-19)
        #print(temp_value)
        if temp_value > max_value:
            max_value = temp_value
            max_action = i
            
    next_state = get_constant_wind_state(state,board,max_action,wind_list)
    print("started at {} and moving to {} with action {} with Q value of {}".format(state,next_state,max_action,max_value))
    state = next_state

started at (0, 3) and moving to (1, 4) with action (1, 1) with Q value of 0.3443496550071051
started at (1, 4) and moving to (2, 5) with action (1, 1) with Q value of 0.3542267144336992
started at (2, 5) and moving to (3, 6) with action (1, 1) with Q value of 0.3686259839414117
started at (3, 6) and moving to (4, 6) with action (1, -1) with Q value of 0.38458532560038544
started at (4, 6) and moving to (5, 6) with action (1, 0) with Q value of 0.39172959731275375
started at (5, 6) and moving to (6, 6) with action (1, 0) with Q value of 0.3982680048955634
started at (6, 6) and moving to (7, 6) with action (1, -1) with Q value of 0.40476474476440183
started at (7, 6) and moving to (8, 6) with action (1, -1) with Q value of 0.41112875932730647
started at (8, 6) and moving to (9, 6) with action (1, 0) with Q value of 0.41748834962536985
started at (9, 6) and moving to (9, 5) with action (0, -1) with Q value of 0.4243209412994594
started at (9, 5) and moving to (9, 4) with action (0, -1) wi

In [147]:
for key,value in Q.items():
    #if value >= 0:
    #    print(key,value)
    if key[0] == (7,3):
        print(key,value)